# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [158]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

from geo_key import geoapify_key


In [159]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bredasdorp,-34.5322,20.0403,65.19,91,2,4.61,ZA,1707714216
1,1,codrington,-38.2667,141.9667,72.66,58,0,11.48,AU,1707714216
2,2,jamestown,42.0970,-79.2353,32.02,90,100,5.01,US,1707714053
3,3,yellowknife,62.4560,-114.3525,5.56,96,20,5.75,CA,1707714073
4,4,olonkinbyen,70.9221,-8.7187,28.00,64,100,10.07,SJ,1707714216


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [160]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [161]:
# Narrow down cities that fit criteria and drop any results with null values
#ideal_cities = (city_data_df["Max Temp"] >= 65) & (city_data_df["Max Temp"] <= 85)

#ideal_cities = city_data_df.query('Max Temp > 65 and Humidity < 70')
ideal_cities = city_data_df.loc[(city_data_df["Max Temp"] >= 65) 
                                & (city_data_df["Max Temp"] <= 85) 
                                & (city_data_df["Humidity"] <= 50)
                                & (city_data_df["Humidity"] >= 20)
                                & (city_data_df["Cloudiness"] <= 50)
                                & (city_data_df["Wind Speed"] <= 15)]
# Drop any rows with null values
ideal_cities.dropna(inplace = True)

# Display sample data
ideal_cities

C:\Users\nemuk\AppData\Local\Temp\ipykernel_25320\1272430606.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_cities.dropna(inplace = True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
15,15,tura,25.5198,90.2201,78.75,24,1,3.89,IN,1707714218
135,135,tambacounda,13.7539,-13.7586,75.25,29,0,6.91,SN,1707714252
138,138,samdrup jongkhar,26.9167,91.6167,69.24,30,0,5.64,BT,1707714253
446,446,boundiali,9.5217,-6.4869,72.72,22,22,3.33,CI,1707714333
483,483,ishigaki,24.3448,124.1572,69.93,43,0,11.50,JP,1707714342
538,538,machali,-34.1833,-70.6667,65.32,50,23,3.62,CL,1707714354


### Step 3: Create a new DataFrame called `hotel_df`.

In [162]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
15,tura,IN,25.5198,90.2201,24,
135,tambacounda,SN,13.7539,-13.7586,29,
138,samdrup jongkhar,BT,26.9167,91.6167,30,
446,boundiali,CI,9.5217,-6.4869,22,
483,ishigaki,JP,24.3448,124.1572,43,
538,machali,CL,-34.1833,-70.6667,50,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [165]:
# Set parameters to search for a hotel
radius = 10000
params = {'categories':'accommodation.hotel',
         'apiKey':geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row['Lng']
    lat = row['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lon},{lat},{radius}'
    params["bias"] = f'proximity:{lon},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    #Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# # Display sample data
hotel_df


Starting hotel search
tura - nearest hotel: No hotel found
tambacounda - nearest hotel: Le relais de Tamba
samdrup jongkhar - nearest hotel: No hotel found
boundiali - nearest hotel: No hotel found
ishigaki - nearest hotel: アートホテル石垣島
machali - nearest hotel: Hotel Boutique Il Giardino


,City,Country,Lat,Lng,Humidity,Hotel Name
15,tura,IN,25.5198,90.2201,24,No hotel found
135,tambacounda,SN,13.7539,-13.7586,29,Le relais de Tamba
138,samdrup jongkhar,BT,26.9167,91.6167,30,No hotel found
446,boundiali,CI,9.5217,-6.4869,22,No hotel found
483,ishigaki,JP,24.3448,124.1572,43,アートホテル石垣島
538,machali,CL,-34.1833,-70.6667,50,Hotel Boutique Il Giardino


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [164]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)